In [1]:
import os
from keras.preprocessing import image

In [2]:
categories = ['with_mask','without_mask']

In [3]:
data = []

for category in categories:
    path = os.path.join('/content/drive/MyDrive/train',category)

    label = categories.index(category)

    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        img = image.load_img(img_path)
        img = image.smart_resize(img,(224,224))
        data.append([img,label])

In [4]:
import random

In [5]:
random.shuffle(data)

In [6]:
X = []
y = []
for features,label in data:
    X.append(features)
    y.append(label)

In [7]:
import numpy as np

In [8]:
X = np.array(X)
y = np.array(y)

In [9]:
print(X.shape,y.shape)

(1279, 224, 224, 3) (1279,)


In [10]:
X = X/255.

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [12]:
from keras.applications.vgg19 import VGG19

In [13]:
vgg = VGG19()

In [14]:
vgg.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
from keras import Sequential

model = Sequential()

In [16]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [17]:
for layer in model.layers:
    layer.trainable = False

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [19]:
from keras.layers import Dense
from keras.callbacks import EarlyStopping

In [20]:
model.add(Dense(1,activation='sigmoid'))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [22]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [23]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)

In [24]:
model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),callbacks=[es])

Epoch 1/20
32/32 [==============================] - 21s 400ms/step - loss: 0.5593 - accuracy: 0.7488 - val_loss: 0.4497 - val_accuracy: 0.8086
Epoch 2/20
32/32 [==============================] - 6s 189ms/step - loss: 0.3496 - accuracy: 0.8886 - val_loss: 0.2659 - val_accuracy: 0.9453
Epoch 3/20
32/32 [==============================] - 6s 187ms/step - loss: 0.2494 - accuracy: 0.9413 - val_loss: 0.2142 - val_accuracy: 0.9531
Epoch 4/20
32/32 [==============================] - 6s 192ms/step - loss: 0.2082 - accuracy: 0.9501 - val_loss: 0.1831 - val_accuracy: 0.9570
Epoch 5/20
32/32 [==============================] - 6s 191ms/step - loss: 0.1822 - accuracy: 0.9531 - val_loss: 0.1593 - val_accuracy: 0.9688
Epoch 6/20
32/32 [==============================] - 6s 193ms/step - loss: 0.1594 - accuracy: 0.9609 - val_loss: 0.1454 - val_accuracy: 0.9688
Epoch 7/20
32/32 [==============================] - 6s 191ms/step - loss: 0.1441 - accuracy: 0.9687 - val_loss: 0.1470 - val_accuracy: 0.9609
Epoch

In [25]:
# prompt: import pickle and pickle the model
import pickle
with open('model.pkl', 'wb') as f:
  pickle.dump(model, f)